# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [1]:
import pandas as pd
import requests
from geopy import distance

In [2]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php
# TODO change to all gb stations so it is not london centric

london_stations = pd.read_csv('london_stations.csv')

In [3]:
london_stations.head()

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508758,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516887,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN


In [4]:
# create list of tuples for use in end function
stations_tuples_list = []
for x in london_stations[['Station', 'Latitude', 'Longitude']].values:
    stations_tuples_list.append((x[0],(x[1],x[2])))

In [5]:
test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [6]:
# 1. Geocode postcode data using api

# create endpoint
def create_endpoint(postcode):
    postcode = postcode.replace(' ','').lower()
    output = f'http://api.postcodes.io/postcodes/{postcode}'
    return output

In [7]:
create_endpoint(test_postcodes[0])

'http://api.postcodes.io/postcodes/n153ad'

In [8]:
# Get longitude and latitude for the code
def get_long_lat(postcode):
    endpoint = create_endpoint(postcode)
    output_dict = {}
    r = requests.get(endpoint)
    output_dict['postcode'] = r.json()['result']['postcode']
    output_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
    return output_dict

In [9]:
test_dict = get_long_lat(test_postcodes[0])

In [10]:
test_dict

{'postcode': 'N15 3AD', 'lat_lon': (51.583631, -0.092808)}

In [11]:
# measure distance between two points

# test with postcode
# distance.distance((lat, lon), (lat, lon))

print(distance.distance(test_dict['lat_lon'], (51.587998, -0.060188)).miles)
print(distance.distance(test_dict['lat_lon'], (51.587998, -0.060188)).km)

1.4369277331214914
2.3125110257326735


In [12]:
# TODO Could split this into two functions, one to retrieve an list of stations ranked in order or closeness
# second one, just to grab the nearest
# TODO round to 1 decimal place
def find_nearest_london_station(postcode, stations_list):
    pc_lat_lon = get_long_lat(postcode)['lat_lon']
    distances_in_m = [(x[0],distance.distance(pc_lat_lon, x[1]).miles) for x in stations_list]
    distances_in_m.sort(key=lambda x:x[1])
    return(distances_in_m[0])

In [13]:
find_nearest_london_station('x', stations_tuples_list)

('Silver Street', 0.510218103915083)

In [14]:
find_nearest_london_station('x', stations_tuples_list)

('Turnpike Lane', 0.15053897620183948)